[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/FER_WINO_SEPARABLE_NO_CUDNN.ipynb)

In [3]:
!wget https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/download/fer2013.tar.gz

--2018-06-18 14:58:07--  https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/download/fer2013.tar.gz
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?ReturnUrl=%2fc%2fchallenges-in-representation-learning-facial-expression-recognition-challenge%2fdownload%2ffer2013.tar.gz [following]
--2018-06-18 14:58:07--  https://www.kaggle.com/account/login?ReturnUrl=%2fc%2fchallenges-in-representation-learning-facial-expression-recognition-challenge%2fdownload%2ffer2013.tar.gz
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 8229 (8.0K) [text/html]
Saving to: ‘fer2013.tar.gz.1’

fer2013.tar.gz.1    100%[===================>]   8.04K  --.-KB/s    in 0s      

2018-06-18 14:58:08 (125 MB/s) - ‘fer2013.tar.gz.1’ saved [8229/8229]



In [4]:
from google.colab import files
uploaded = files.upload()

Saving fer2013.tar to fer2013.tar


In [1]:
#!pip uninstall tensorflow-gpu -y
#!pip install tensorflow
!rm fer2013*
!ls

datalab


In [42]:
import os
!export TF_ENABLE_WINOGRAD_NONFUSED=1
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'
os.environ.pop('TF_ENABLE_WINOGRAD_NONFUSED', None)

'1'

In [5]:
!tar xvf fer2013.tar
!ls

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/
datalab  fer2013  fer2013.tar  fer2013.tar.gz  fer2013.tar.gz.1


In [6]:

import csv
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.layers import SeparableConv2D, MaxPooling2D, BatchNormalization 
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
img_rows, img_cols = 48, 48
batch_size = 128
nb_classes = 7
nb_epoch = 1200
img_channels = 1

In [0]:
import csv
f = open('fer2013/fer2013.csv')
csv_f = csv.reader(f)

train_x = []
train_y = []
val_x =[]
val_y =[]


In [9]:
print(csv_f)

In [0]:
num=0
for row in csv_f:
  #print(row)
  num = num +1
  #print(num)
  temp_list = []
  for pixel in row[1].split( ):
    temp_list.append(int(pixel))

  if str(row[2]) == "Training":
    train_y.append(int(row[0]))
    train_x.append(temp_list) 
  elif str(row[2]) == "PublicTest":
    val_y.append(int(row[0]))
    val_x.append(temp_list)

In [0]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)

In [0]:
train_x = train_x.reshape(train_x.shape[0], 48, 48)
train_x = train_x.reshape(train_x.shape[0], 1, 48, 48)
train_y = np_utils.to_categorical(train_y, 7)

In [0]:
val_x = val_x.reshape(val_x.shape[0], 48, 48)
val_x = val_x.reshape(val_x.shape[0], 1, 48, 48)
val_y = np_utils.to_categorical(val_y, 7)

In [0]:
train_x = train_x.astype('float32')
train_x = train_x / 255.0
val_x = val_x.astype('float32')
val_x = val_x / 255.0


In [0]:
input_shape = (img_channels, img_rows, img_cols)
model = Sequential()
model.add(SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same',
                            name='image_array', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

model.add(SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.2))

model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(SeparableConv2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(Flatten())

model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.45))

model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.45))

model.add(Dense(7))
model.add(Activation('softmax'))

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (SeparableConv2D (None, 1, 48, 64)         3568      
_________________________________________________________________
batch_normalization_41 (Batc (None, 1, 48, 64)         256       
_________________________________________________________________
activation_45 (Activation)   (None, 1, 48, 64)         0         
_________________________________________________________________
separable_conv2d_29 (Separab (None, 1, 48, 64)         4736      
_________________________________________________________________
batch_normalization_42 (Batc (None, 1, 48, 64)         256       
_________________________________________________________________
activation_46 (Activation)   (None, 1, 48, 64)         0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 1, 24, 64)         0         
__________

In [0]:
model.compile(optimizer='Adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [46]:
# Wino 1 Separable
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 162s 6ms/step - loss: 2.0319 - acc: 0.2314 - val_loss: 1.7890 - val_acc: 0.2611
Epoch 2/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.8674 - acc: 0.2725 - val_loss: 1.9301 - val_acc: 0.2909
Epoch 3/10
  896/28709 [..............................] - ETA: 2:26 - loss: 1.8165 - acc: 0.2846

28709/28709 [==============================] - 157s 5ms/step - loss: 1.7964 - acc: 0.3042 - val_loss: 1.9084 - val_acc: 0.3305
Epoch 4/10
28709/28709 [==============================] - 158s 6ms/step - loss: 1.7450 - acc: 0.3279 - val_loss: 3.7197 - val_acc: 0.2956
Epoch 5/10
10880/28709 [==========>...................] - ETA: 1:34 - loss: 1.7126 - acc: 0.3461

28709/28709 [==============================] - 158s 6ms/step - loss: 1.7074 - acc: 0.3482 - val_loss: 1.6456 - val_acc: 0.3586
Epoch 6/10
28709/28709 [==============================] - 158s 6ms/step - loss: 1.6646 - acc: 0.3627 - val_loss: 1.7632 - val_acc: 0.3402
Epoch 7/10
13184/28709 [============>.................] - ETA: 1:22 - loss: 1.6483 - acc: 0.3651

28709/28709 [==============================] - 158s 6ms/step - loss: 1.6387 - acc: 0.3711 - val_loss: 1.6552 - val_acc: 0.3650
Epoch 8/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.6062 - acc: 0.3847 - val_loss: 1.5471 - val_acc: 0.3892
Epoch 9/10
13824/28709 [=============>................] - ETA: 1:19 - loss: 1.5900 - acc: 0.3901

28709/28709 [==============================] - 157s 5ms/step - loss: 1.5774 - acc: 0.3942 - val_loss: 1.5438 - val_acc: 0.4057
Epoch 10/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.5582 - acc: 0.4022 - val_loss: 1.5518 - val_acc: 0.3976
--- 1581.2885239124298 seconds ---


In [50]:
# input normalized
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 162s 6ms/step - loss: 2.0189 - acc: 0.2407 - val_loss: 1.8390 - val_acc: 0.2856
Epoch 2/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.8329 - acc: 0.2909 - val_loss: 1.7546 - val_acc: 0.3391
Epoch 3/10
  896/28709 [..............................] - ETA: 2:27 - loss: 1.7662 - acc: 0.3058

28709/28709 [==============================] - 157s 5ms/step - loss: 1.7695 - acc: 0.3230 - val_loss: 1.8986 - val_acc: 0.3093
Epoch 4/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.7122 - acc: 0.3484 - val_loss: 1.6657 - val_acc: 0.3611
Epoch 5/10
10880/28709 [==========>...................] - ETA: 1:34 - loss: 1.6901 - acc: 0.3509

28709/28709 [==============================] - 156s 5ms/step - loss: 1.6773 - acc: 0.3604 - val_loss: 1.7014 - val_acc: 0.3494
Epoch 6/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.6553 - acc: 0.3702 - val_loss: 1.6194 - val_acc: 0.3859
Epoch 7/10
13312/28709 [============>.................] - ETA: 1:20 - loss: 1.6141 - acc: 0.3857

28709/28709 [==============================] - 156s 5ms/step - loss: 1.6129 - acc: 0.3857 - val_loss: 1.6001 - val_acc: 0.3923
Epoch 8/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.5920 - acc: 0.3924 - val_loss: 1.6594 - val_acc: 0.4035
Epoch 9/10
13824/28709 [=============>................] - ETA: 1:18 - loss: 1.5641 - acc: 0.4065

28709/28709 [==============================] - 157s 5ms/step - loss: 1.5559 - acc: 0.4044 - val_loss: 1.5507 - val_acc: 0.4113
Epoch 10/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.5341 - acc: 0.4100 - val_loss: 1.5732 - val_acc: 0.4068
--- 1574.3259291648865 seconds ---


In [0]:
filepath='Model.{epoch:02d}-{val_acc:.4f}.hdf5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')

import time
start_time = time.time()

datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=True,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=True,  # divide each input by its std
    zca_whitening=True,  # apply ZCA whitening
    rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

datagen.fit(train_x)

model.fit_generator(datagen.flow(train_x, train_y,
                    batch_size=batch_size),
                    steps_per_epoch=(train_x.shape[0]/batch_size),
                    epochs=10,
                    validation_data=(val_x, val_y),
                    callbacks=[checkpointer])
print("--- %s seconds ---" % (time.time() - start_time))

In [29]:
# Wino 1 Separable
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 13s 459us/step - loss: 2.0133 - acc: 0.2504 - val_loss: 1.7558 - val_acc: 0.2931
Epoch 2/10
28709/28709 [==============================] - 10s 350us/step - loss: 1.8040 - acc: 0.3142 - val_loss: 1.7831 - val_acc: 0.3232
Epoch 3/10
28709/28709 [==============================] - 10s 345us/step - loss: 1.6975 - acc: 0.3585 - val_loss: 1.7235 - val_acc: 0.3709
Epoch 4/10
  128/28709 [..............................] - ETA: 9s - loss: 1.8020 - acc: 0.3828

28709/28709 [==============================] - 10s 347us/step - loss: 1.6248 - acc: 0.3901 - val_loss: 1.6815 - val_acc: 0.3837
Epoch 5/10
28709/28709 [==============================] - 10s 346us/step - loss: 1.5606 - acc: 0.4072 - val_loss: 1.6298 - val_acc: 0.4107
Epoch 6/10
28709/28709 [==============================] - 10s 345us/step - loss: 1.5046 - acc: 0.4302 - val_loss: 1.5186 - val_acc: 0.4185
Epoch 7/10
10624/28709 [==========>...................] - ETA: 6s - loss: 1.4488 - acc: 0.4547

28709/28709 [==============================] - 10s 350us/step - loss: 1.4610 - acc: 0.4486 - val_loss: 1.5716 - val_acc: 0.4269
Epoch 8/10
28709/28709 [==============================] - 10s 350us/step - loss: 1.4148 - acc: 0.4682 - val_loss: 1.4896 - val_acc: 0.4366
Epoch 9/10
28709/28709 [==============================] - 10s 348us/step - loss: 1.3785 - acc: 0.4808 - val_loss: 1.4299 - val_acc: 0.4425
Epoch 10/10
12800/28709 [============>.................] - ETA: 5s - loss: 1.3371 - acc: 0.4895

28709/28709 [==============================] - 10s 349us/step - loss: 1.3455 - acc: 0.4874 - val_loss: 1.4069 - val_acc: 0.4595
--- 105.74680256843567 seconds ---


In [36]:
# Wino 1 Regular
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=5, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 21s 715us/step - loss: 1.9565 - acc: 0.2723 - val_loss: 1.7349 - val_acc: 0.3266
Epoch 2/5
27648/28709 [===========================>..] - ETA: 0s - loss: 1.7632 - acc: 0.3394

28709/28709 [==============================] - 15s 533us/step - loss: 1.7624 - acc: 0.3405 - val_loss: 2.6360 - val_acc: 0.2577
Epoch 3/5
28709/28709 [==============================] - 15s 536us/step - loss: 1.6694 - acc: 0.3788 - val_loss: 1.7269 - val_acc: 0.3901
Epoch 4/5
15360/28709 [===============>..............] - ETA: 6s - loss: 1.6046 - acc: 0.4005

28709/28709 [==============================] - 15s 531us/step - loss: 1.6014 - acc: 0.3990 - val_loss: 1.6849 - val_acc: 0.3650
Epoch 5/5
28709/28709 [==============================] - 15s 536us/step - loss: 1.5383 - acc: 0.4241 - val_loss: 1.5914 - val_acc: 0.4004
--- 84.23344898223877 seconds ---


In [41]:
# Wino 0 Regular
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=5, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 19s 666us/step - loss: 1.9578 - acc: 0.2694 - val_loss: 1.7939 - val_acc: 0.3073
Epoch 2/5
28544/28709 [============================>.] - ETA: 0s - loss: 1.7300 - acc: 0.3487

28709/28709 [==============================] - 15s 536us/step - loss: 1.7292 - acc: 0.3488 - val_loss: 1.6637 - val_acc: 0.3452
Epoch 3/5
28709/28709 [==============================] - 15s 535us/step - loss: 1.6358 - acc: 0.3831 - val_loss: 2.1404 - val_acc: 0.2792
Epoch 4/5
15104/28709 [==============>...............] - ETA: 7s - loss: 1.6028 - acc: 0.4033

28709/28709 [==============================] - 15s 533us/step - loss: 1.5859 - acc: 0.4056 - val_loss: 1.6214 - val_acc: 0.3773
Epoch 5/5
28709/28709 [==============================] - 15s 534us/step - loss: 1.5197 - acc: 0.4234 - val_loss: 1.5772 - val_acc: 0.3996
--- 82.82288789749146 seconds ---


In [46]:
# Wino 1 Regular
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=5, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 20s 681us/step - loss: 1.9462 - acc: 0.2755 - val_loss: 1.7434 - val_acc: 0.3266
Epoch 2/5
27648/28709 [===========================>..] - ETA: 0s - loss: 1.7649 - acc: 0.3430

28709/28709 [==============================] - 15s 538us/step - loss: 1.7648 - acc: 0.3434 - val_loss: 1.9530 - val_acc: 0.3380
Epoch 3/5
28709/28709 [==============================] - 15s 537us/step - loss: 1.6668 - acc: 0.3793 - val_loss: 1.5976 - val_acc: 0.3639
Epoch 4/5
15360/28709 [===============>..............] - ETA: 6s - loss: 1.6099 - acc: 0.4033

28709/28709 [==============================] - 15s 534us/step - loss: 1.5981 - acc: 0.4060 - val_loss: 1.6072 - val_acc: 0.3915
Epoch 5/5
28709/28709 [==============================] - 15s 536us/step - loss: 1.5397 - acc: 0.4279 - val_loss: 1.6131 - val_acc: 0.4115
--- 83.71084761619568 seconds ---


In [50]:
# Wino 1 Regular 1x3 3x1
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 12s 403us/step - loss: 1.6001 - acc: 0.3919 - val_loss: 1.8056 - val_acc: 0.3633
Epoch 2/10
28709/28709 [==============================] - 12s 404us/step - loss: 1.5769 - acc: 0.4010 - val_loss: 1.5454 - val_acc: 0.3968
Epoch 3/10
 8320/28709 [=======>......................] - ETA: 8s - loss: 1.5557 - acc: 0.4109

28709/28709 [==============================] - 12s 408us/step - loss: 1.5528 - acc: 0.4049 - val_loss: 1.5312 - val_acc: 0.3945
Epoch 4/10
28709/28709 [==============================] - 12s 409us/step - loss: 1.5132 - acc: 0.4224 - val_loss: 1.5086 - val_acc: 0.4068
Epoch 5/10
17920/28709 [=================>............] - ETA: 4s - loss: 1.4812 - acc: 0.4359

28709/28709 [==============================] - 12s 409us/step - loss: 1.4866 - acc: 0.4333 - val_loss: 1.4970 - val_acc: 0.4099
Epoch 6/10
28709/28709 [==============================] - 12s 409us/step - loss: 1.4692 - acc: 0.4366 - val_loss: 1.4706 - val_acc: 0.4361
Epoch 7/10
21120/28709 [=====================>........] - ETA: 2s - loss: 1.4426 - acc: 0.4434

28709/28709 [==============================] - 12s 406us/step - loss: 1.4473 - acc: 0.4439 - val_loss: 1.4631 - val_acc: 0.4330
Epoch 8/10
28709/28709 [==============================] - 12s 410us/step - loss: 1.4302 - acc: 0.4500 - val_loss: 1.4582 - val_acc: 0.4405
Epoch 9/10
22272/28709 [======================>.......] - ETA: 2s - loss: 1.4148 - acc: 0.4572

28709/28709 [==============================] - 12s 408us/step - loss: 1.4161 - acc: 0.4561 - val_loss: 1.4536 - val_acc: 0.4327
Epoch 10/10
28709/28709 [==============================] - 12s 402us/step - loss: 1.3887 - acc: 0.4660 - val_loss: 1.4843 - val_acc: 0.4372
--- 116.78946256637573 seconds ---
